# Match file

In [36]:
import pandas as pd
import numpy as np
import os
import re
import math

pd.set_option('display.height', 300)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

height has been deprecated.



# Evaluation

In [2]:
Datapath='./../Dataeva/'
evaluationfile=Datapath+'evaluationn.csv'
dfe=pd.read_csv(evaluationfile,sep=',')
#add sequence label(entrainement-0,match1-1,match2-2,match3-3)+(1°mitemps-1, 2°mitemps-2) 01,02
dfe.ix[dfe['Numéro du Match']=='entrainement',2]=0
dfe.ix[dfe['Quelle mi-temps ?']=='1° Mi-temps',3]=1
dfe.ix[dfe['Quelle mi-temps ?']=='2° Mi-temps',3]=2
dfe['seqlabel']=dfe.apply(lambda row: str(row['Numéro du Match'])+str(row['Quelle mi-temps ?']),axis=1)
#Add Subject label: S2-S58
dfe['sujet']=pd.Series(['S2']*6+map(lambda t: t[0]+str(t[1]), 
    zip(['S']*56*8,[i for i in range(3,59) for _ in range(8)])))

dfe.columns = ['horodateur', 'nom','NoMatch','Nomitemps',
               'noteg','q1','q2','q3','q4','q5','q6','q7','q8',
               'change','emotion','noteI','noteD','noteA','eventchange','quelevent','seqlabel','sujet']
dfe.tail(12)

,horodateur,nom,NoMatch,Nomitemps,noteg,q1,q2,q3,q4,q5,...,q8,change,emotion,noteI,noteD,noteA,eventchange,quelevent,seqlabel,sujet
442,24/06/2016 14:33:07,SAMI Malick,2,1,3,Pas vraiment,Pas du tout,Plutôt,Pas vraiment,Plutôt,...,Pas du tout,Non,"Colère, Frustration",5,5,4,Non,NaN,21,S57
443,24/06/2016 14:40:01,SAMI Malick,2,2,2,Pas vraiment,Pas du tout,Beaucoup,Plutôt,Plutôt,...,Pas vraiment,Non,"Colère, Frustration",4,5,4,Non,NaN,22,S57
444,24/06/2016 14:59:55,SAMI Malick,3,1,4,Plutôt,Pas du tout,Plutôt,Plutôt,Pas vraiment,...,Un peu,Oui,"Frustration, ennui",5,2,4,Oui,Manque d'occasions de but,31,S57
445,24/06/2016 15:07:31,SAMI Malick,3,2,4,Plutôt,Un peu,Beaucoup,Plutôt,Pas vraiment,...,Pas du tout,Oui,"Joie, Colère, Frustration",5,2,4,Oui,tirs ratés devant le but,32,S57
446,24/06/2016 16:06:19,Grandidier Alex,0,1,5,Pas vraiment,Pas du tout,Plutôt,Pas vraiment,Plutôt,...,Pas vraiment,Oui,"Fierté, Frustration",3,3,2,Oui,Réussite d'un exercice après plusieurs échecs,01,S58
447,24/06/2016 16:12:16,Grandidier Alex,0,2,7,Pas vraiment,Un peu,Un peu,Plutôt,Un peu,...,Pas vraiment,Non,"Joie, Fierté, Frustration",4,2,3,Non,NaN,02,S58
448,24/06/2016 16:23:42,Grandidier Alex,1,1,6,Plutôt,Pas vraiment,Plutôt,Plutôt,Un peu,...,Pas vraiment,Non,"Joie, Colère, Frustration",3,2,3,Non,NaN,11,S58
449,24/06/2016 16:29:59,Grandidier Alex,1,2,6,Un peu,Un peu,Plutôt,Plutôt,Un peu,...,Pas vraiment,Non,"Colère, Frustration",4,3,3,Non,NaN,12,S58
450,24/06/2016 16:52:10,Grandidier Alex,2,1,7,Plutôt,Un peu,Pas vraiment,Plutôt,Un peu,...,Pas du tout,Non,"Peur, Joie, Frustration, soulagement",4,3,3,Non,NaN,21,S58
451,24/06/2016 16:58:59,Grandidier Alex,2,2,4,Un peu,Pas du tout,Plutôt,Plutôt,Un peu,...,Pas vraiment,Non,"Peur, Colère, Frustration, soulagement",4,3,3,Non,NaN,22,S58


In [3]:
def evanote(option):
    return {
        'Pas du tout': 1,
        'Pas vraiment': 2,
        'Un peu':3,
        'Plutôt': 4,
        'Beaucoup': 5,
    }[option]

def qnote(row):
    #amu=(evanote(row['q1'])+evanote(row['q4'])+(6-evanote(row['q8'])))/3.0
    amu=(evanote(row['q1'])+evanote(row['q4']))/2.0
    diffi= ((6-evanote(row['q2']))+evanote(row['q5']))/2.0
    #impli=(evanote(row['q6'])+(6-evanote(row['q7']))+ (6-evanote(row['q8'])))/3.0
    impli=(evanote(row['q6'])+(6-evanote(row['q7'])))/2.0
    return (amu,diffi,impli)
#q1: joyeux
#q2: bon
#q3: frustré
#q4: amusant
#q5: difficile
#q6: concentré
#q7: autre chose
#q8: ennuyé

In [4]:
dfe['qnote']=dfe.apply (lambda row: qnote(row),axis=1)
dfe['qI']=dfe.apply(lambda row: row['qnote'][2],axis=1)
dfe['qD']=dfe.apply(lambda row: row['qnote'][1],axis=1)
dfe['qA']=dfe.apply(lambda row: row['qnote'][0],axis=1)
dfe.ix[dfe['eventchange']=='Oui','eventchange']=1
dfe.ix[dfe['eventchange']=='Non','eventchange']=0
dfe.ix[dfe['change']=='Oui','change']=1
dfe.ix[dfe['change']=='Non','change']=0
dfe.ix[dfe['change']=='Peut-être','change']=-1
evadf=dfe[['sujet','seqlabel','NoMatch','Nomitemps','noteg','change','eventchange','noteI','noteD','noteA','qI','qD','qA']]
evadf

,sujet,seqlabel,NoMatch,Nomitemps,noteg,change,eventchange,noteI,noteD,noteA,qI,qD,qA
0,S2,11,1,1,5,-1,1,3,2,3,4.0,3.0,2.5
1,S2,12,1,2,6,1,1,3,2,3,4.0,3.5,3.5
2,S2,21,2,1,4,0,0,3,3,2,4.0,3.5,2.0
3,S2,22,2,2,4,1,1,4,3,3,4.0,4.0,2.5
4,S2,31,3,1,6,1,1,4,1,3,2.0,2.5,3.0
5,S2,32,3,2,6,1,1,4,1,3,4.0,2.0,3.0
6,S3,01,0,1,5,1,0,5,3,4,5.0,3.5,2.0
7,S3,02,0,2,4,1,0,5,5,2,5.0,5.0,1.5
8,S3,11,1,1,2,0,1,5,5,3,5.0,4.5,2.0
9,S3,12,1,2,6,1,0,5,4,4,5.0,3.5,3.5


# Evaluation global

In [5]:
Datapath='./../Dataeva/'
evaluationfile=Datapath+'evaglobal.csv'
dfg1=pd.read_csv(evaluationfile,sep=',')
l=len(dfg1)
dfg1['sujet']=pd.Series(
    map(lambda t: t[0]+str(t[1]),
    zip(['S']*l,range(1,l+1))
       )
    )
dfg1['D']=dfg1['Difficulté'].map(lambda x: x.split('>'))
dfg1['I']=dfg1['Implication'].map(lambda x: x.split('>'))
dfg1['A']=dfg1['Amusement'].map(lambda x: x.split('>'))
dfg1



,Horodateur,Nom/prénom,Difficulté,Implication,Amusement,sujet,D,I,A
0,09/05/2016 14:38:18,Cyrille KAMDEM,1>0>2>3,3>0>2>1,3>0>2>1,S1,"[1, 0, 2, 3]","[3, 0, 2, 1]","[3, 0, 2, 1]"
1,11/05/2016 12:02:11,Rhouzlane Elias,1>0>2>3,2>0>1>3,2>0>3>1,S2,"[1, 0, 2, 3]","[2, 0, 1, 3]","[2, 0, 3, 1]"
2,12/05/2016 15:30:33,Cairoche Gabrielle,1>0>2>3,1>0>3>2,3>1>0>2,S3,"[1, 0, 2, 3]","[1, 0, 3, 2]","[3, 1, 0, 2]"
3,12/05/2016 18:18:19,GOMEZ DAVID,3>1>0>2,3>2>1>0,3>2>1>0,S4,"[3, 1, 0, 2]","[3, 2, 1, 0]","[3, 2, 1, 0]"
4,13/05/2016 12:10:13,RABIA,1>0>3>2,2>3>1>0,3>2>1>0,S5,"[1, 0, 3, 2]","[2, 3, 1, 0]","[3, 2, 1, 0]"
5,13/05/2016 15:34:20,Démétrius/David,1>2>3>0,0>1>2>3,3>2>1>0,S6,"[1, 2, 3, 0]","[0, 1, 2, 3]","[3, 2, 1, 0]"
6,13/05/2016 18:07:01,LE Dao Khanh Vu,2>3>1>0,1>0>3>2,1>0>3>2,S7,"[2, 3, 1, 0]","[1, 0, 3, 2]","[1, 0, 3, 2]"
7,20/05/2016 12:02:43,Dany Naomi,2>0>1>3,0>1>2>3,1>3>2>0,S8,"[2, 0, 1, 3]","[0, 1, 2, 3]","[1, 3, 2, 0]"
8,20/05/2016 15:10:28,kama / babacar,1>2>3>0,1>2>3>0,3>1>2>0,S9,"[1, 2, 3, 0]","[1, 2, 3, 0]","[3, 1, 2, 0]"
9,20/05/2016 17:55:38,Fontaine emile,0>1>2>3,1>2>3>0,2>0>1>3,S10,"[0, 1, 2, 3]","[1, 2, 3, 0]","[2, 0, 1, 3]"


In [6]:
Datapath='./../Dataeva/'
evaluationfile=Datapath+'evaglobal1.csv'
dfg2=pd.read_csv(evaluationfile,sep=',')
l2=len(dfg2)
dfg2['sujet']=pd.Series(
    map(lambda t:t[0]+str(t[1]),zip(['S']*l2,range(18,18+l2)))
)
tmp_diffi=dfg2[['sujet','D0','D1','D2','D3']]
s_diffi=pd.melt(tmp_diffi,id_vars=['sujet']).pivot(index='sujet', columns='value', values='variable').reset_index()
dfg2['D']=s_diffi.apply(lambda row: [int(row[1].split('D')[1]),int(row[2].split('D')[1]),int(row[3].split('D')[1]),int(row[4].split('D')[1])]
              ,axis=1)

tmp_impli=dfg2[['sujet','I0','I1','I2','I3']]
s_impli=pd.melt(tmp_impli,id_vars=['sujet']).pivot(index='sujet', columns='value', values='variable').reset_index()
dfg2['I']=s_impli.apply(lambda row: [int(row[1].split('I')[1]),int(row[2].split('I')[1]),int(row[3].split('I')[1]),int(row[4].split('I')[1])]
              ,axis=1)

tmp_amu=dfg2[['sujet','A0','A1','A2','A3']]
s_amu=pd.melt(tmp_amu,id_vars=['sujet']).pivot(index='sujet', columns='value', values='variable').reset_index()
dfg2['A']=s_amu.apply(lambda row: [int(row[1].split('A')[1]),int(row[2].split('A')[1]),int(row[3].split('A')[1]),int(row[4].split('A')[1])]
              ,axis=1)
tmp_exci=dfg2[['sujet','E0','E1','E2','E3']]
s_exci=pd.melt(tmp_exci,id_vars=['sujet']).pivot(index='sujet', columns='value', values='variable').reset_index()
dfg2['E']=s_exci.apply(lambda row: [int(row[1].split('E')[1]),int(row[2].split('E')[1]),int(row[3].split('E')[1]),int(row[4].split('E')[1])]
              ,axis=1)

In [7]:
dfg=dfg1[['sujet','D','I','A']].append(dfg2[['sujet','D','I','A']],ignore_index=True)
dfg

,sujet,D,I,A
0,S1,"[1, 0, 2, 3]","[3, 0, 2, 1]","[3, 0, 2, 1]"
1,S2,"[1, 0, 2, 3]","[2, 0, 1, 3]","[2, 0, 3, 1]"
2,S3,"[1, 0, 2, 3]","[1, 0, 3, 2]","[3, 1, 0, 2]"
3,S4,"[3, 1, 0, 2]","[3, 2, 1, 0]","[3, 2, 1, 0]"
4,S5,"[1, 0, 3, 2]","[2, 3, 1, 0]","[3, 2, 1, 0]"
5,S6,"[1, 2, 3, 0]","[0, 1, 2, 3]","[3, 2, 1, 0]"
6,S7,"[2, 3, 1, 0]","[1, 0, 3, 2]","[1, 0, 3, 2]"
7,S8,"[2, 0, 1, 3]","[0, 1, 2, 3]","[1, 3, 2, 0]"
8,S9,"[1, 2, 3, 0]","[1, 2, 3, 0]","[3, 1, 2, 0]"
9,S10,"[0, 1, 2, 3]","[1, 2, 3, 0]","[2, 0, 1, 3]"


In [8]:
def rank1(metric):
    dfg['1']=dfg.apply(lambda row: row[metric][0],axis=1)
    dfg['2']=dfg.apply(lambda row: row[metric][1],axis=1)
    dfg['3']=dfg.apply(lambda row: row[metric][2],axis=1)
    dfg['4']=dfg.apply(lambda row: row[metric][3],axis=1)
    return dfg[['sujet','1','2','3','4']]
def sujetnum(sujetname):
    return sujetname

In [9]:
rankevadf=evadf[(evadf['Nomitemps']==2) & (evadf['sujet']!='S2')]
rnI=pd.DataFrame({'rnI':(rankevadf.groupby('sujet')['noteI'].rank(ascending=False))})
rnD=pd.DataFrame({'rnD':(rankevadf.groupby('sujet')['noteD'].rank(ascending=False))})
rnA=pd.DataFrame({'rnA':(rankevadf.groupby('sujet')['noteA'].rank(ascending=False))})
rqI=pd.DataFrame({'rqI':(rankevadf.groupby('sujet')['qI'].rank(ascending=False))})
rqD=pd.DataFrame({'rqD':(rankevadf.groupby('sujet')['qD'].rank(ascending=False))})
rqA=pd.DataFrame({'rqA':(rankevadf.groupby('sujet')['qA'].rank(ascending=False))})

rankevadf=rankevadf.join([rnI,rnD,rnA,rqI,rqD,rqA])
rankevadf['NoMatch']=rankevadf['NoMatch'].astype(str)

In [10]:
implicationrank=rank1('I')
difficilerank=rank=rank1('D')
amusantrank=rank1('A')
amusantrank.head()

,sujet,1,2,3,4
0,S1,3,0,2,1
1,S2,2,0,3,1
2,S3,3,1,0,2
3,S4,3,2,1,0
4,S5,3,2,1,0


In [11]:

tmpI=pd.melt(implicationrank,id_vars=['sujet'], value_vars=['1','2','3','4']).sort(['sujet','value'])
tmpI.columns=['sujet','rankgI','NoMatch']
tmpI['NoMatch']=tmpI['NoMatch'].astype(str)

tmpD=pd.melt(implicationrank,id_vars=['sujet'], value_vars=['1','2','3','4']).sort(['sujet','value'])
tmpD.columns=['sujet','rankgD','NoMatch']
tmpD['NoMatch']=tmpD['NoMatch'].astype(str)

tmpA=pd.melt(implicationrank,id_vars=['sujet'], value_vars=['1','2','3','4']).sort(['sujet','value'])
tmpA.columns=['sujet','rankgA','NoMatch']
tmpA['NoMatch']=tmpA['NoMatch'].astype(str)

rankevadf=rankevadf.merge(tmpI, on=['sujet','NoMatch'],how='left')
rankevadf=rankevadf.merge(tmpD, on=['sujet','NoMatch'],how='left')
rankevadf=rankevadf.merge(tmpA, on=['sujet','NoMatch'],how='left')

#### Evaluation of correlation of ranking and auto-evaluation

In [12]:
rankevadf['rdiffqI']=rankevadf.apply(lambda row: np.fabs(float(row['rankgI'])-float(row['rqI'])),axis=1)
rankevadf['rdiffqD']=rankevadf.apply(lambda row: np.fabs(float(row['rankgD'])-float(row['rqD'])),axis=1)
rankevadf['rdiffqA']=rankevadf.apply(lambda row: np.fabs(float(row['rankgA'])-float(row['rqA'])),axis=1)

rankevadf['rdiffnI']=rankevadf.apply(lambda row: np.fabs(float(row['rankgI'])-float(row['rnI'])),axis=1)
rankevadf['rdiffnD']=rankevadf.apply(lambda row: np.fabs(float(row['rankgD'])-float(row['rnD'])),axis=1)
rankevadf['rdiffnA']=rankevadf.apply(lambda row: np.fabs(float(row['rankgA'])-float(row['rnA'])),axis=1)
rankevadf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 217 entries, 0 to 216
Data columns (total 28 columns):
sujet          217 non-null object
seqlabel       217 non-null object
NoMatch        217 non-null object
Nomitemps      217 non-null object
noteg          217 non-null object
change         217 non-null object
eventchange    217 non-null object
noteI          217 non-null int64
noteD          217 non-null int64
noteA          217 non-null int64
qI             217 non-null float64
qD             217 non-null float64
qA             217 non-null float64
rnI            217 non-null float64
rnD            217 non-null float64
rnA            217 non-null float64
rqI            217 non-null float64
rqD            217 non-null float64
rqA            217 non-null float64
rankgI         217 non-null object
rankgD         217 non-null object
rankgA         217 non-null object
rdiffqI        217 non-null float64
rdiffqD        217 non-null float64
rdiffqA        217 non-null float64
rdiffnI    

In [13]:
rankevadf['sujetNum']=rankevadf.apply(lambda row: int(row['sujet'][1:]),axis=1)
rankevadf.to_csv('./../EvaLabel.csv',encoding='utf-16')

In [14]:
rankevadf.head()

,sujet,seqlabel,NoMatch,Nomitemps,noteg,change,eventchange,noteI,noteD,noteA,...,rankgI,rankgD,rankgA,rdiffqI,rdiffqD,rdiffqA,rdiffnI,rdiffnD,rdiffnA,sujetNum
0,S3,02,0,2,4,1,0,5,5,2,...,2,2,2,0.5,1,2,0.5,1.0,1.5,3
1,S3,12,1,2,6,1,0,5,4,4,...,1,1,1,0.5,1,1,0.5,1.5,0.0,3
2,S3,22,2,2,4,1,1,4,4,2,...,4,4,4,0.0,1,1,0.5,1.5,0.5,3
3,S3,32,3,2,7,1,1,4,3,3,...,3,3,3,0.0,1,2,0.5,1.0,1.0,3
4,S4,02,0,2,7,-1,1,3,3,4,...,4,4,4,0.5,1,0,0.0,1.0,0.5,4


In [15]:
rankeva=rankevadf.groupby('sujetNum').agg({'rdiffqI':'sum','rdiffqD':'sum','rdiffqA':'sum','rdiffnI':'sum','rdiffnD':'sum','rdiffnA':'sum',})
rankeva.describe()

,rdiffnD,rdiffnA,rdiffnI,rdiffqI,rdiffqA,rdiffqD
count,56.000000,56.000000,56.000000,56.000000,56.000000,56.000000
mean,4.214286,3.964286,3.696429,3.910714,4.035714,4.375000
std,1.816233,1.715854,1.488375,1.719162,1.916097,2.093822
min,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000
25%,3.000000,3.000000,2.750000,2.750000,2.000000,3.000000
50%,4.000000,4.000000,4.000000,4.000000,4.000000,4.500000
75%,6.000000,5.250000,4.000000,5.000000,6.000000,6.000000
max,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000


In [16]:
rankeva.sort(['rdiffnD','rdiffnA','rdiffnI'])

,rdiffnD,rdiffnA,rdiffnI,rdiffqI,rdiffqA,rdiffqD
sujetNum,,,,,,
9,1,2,1,2,3,2
24,1,4,1,1,3,0
26,1,4,4,6,6,1
58,1,4,4,6,2,1
6,2,4,4,6,4,5
42,2,4,4,1,2,3
36,2,4,6,2,6,1
48,2,5,4,4,4,5
17,2,6,5,6,6,4


In [17]:
dfg2.head()

,Horodateur,Nom/prénom,I0,I1,I2,I3,D0,D1,D2,D3,...,A3,E0,E1,E2,E3,sujet,D,I,A,E
0,27/05/2016 15:16:56,Azizi Yacine,1,3,2,4,3,4,1,2,...,4,2,3,1,4,S18,"[2, 3, 0, 1]","[0, 2, 1, 3]","[0, 2, 1, 3]","[2, 0, 1, 3]"
1,27/05/2016 18:11:31,aznavour alexandre,1,2,3,4,4,3,1,2,...,2,4,3,1,2,S19,"[2, 3, 1, 0]","[0, 1, 2, 3]","[1, 3, 0, 2]","[2, 3, 1, 0]"
2,30/05/2016 18:25:14,Kissié Christopher,2,1,4,3,4,3,1,2,...,3,3,2,4,1,S20,"[2, 3, 1, 0]","[1, 0, 3, 2]","[1, 0, 3, 2]","[3, 1, 0, 2]"
3,31/05/2016 15:27:37,Staiano Jacopo,3,2,1,4,2,3,1,4,...,3,4,1,3,2,S21,"[2, 0, 1, 3]","[2, 1, 0, 3]","[1, 2, 3, 0]","[1, 3, 2, 0]"
4,31/05/2016 17:55:01,SELLE-DELAVAUD Lorraine,2,1,3,4,2,1,3,4,...,4,2,1,3,4,S22,"[1, 0, 2, 3]","[1, 0, 2, 3]","[1, 0, 2, 3]","[1, 0, 2, 3]"


# Sujet & Match statistics

In [18]:
Datapath='./../Dataeva/'
MatchStafile=Datapath+'MatchSta.csv'
mstadf=pd.read_csv(MatchStafile,sep=',')
niveau={
    'debutant':1,
    'amateur':2,
    'semipro':3,
    'pro':4,
    'champion':5,
    'legend':6
}
mstadf.niveau=mstadf.apply(lambda row: niveau[row['niveau']],axis=1)
mstadf.m1_niveau=mstadf.apply(lambda row: niveau[row['m1_niveau']],axis=1)
mstadf.m2_niveau=mstadf.apply(lambda row: niveau[row['m2_niveau']],axis=1)
mstadf.m3_niveau=mstadf.apply(lambda row: niveau[row['m3_niveau']],axis=1)
mstadf.head()
mstadf.columns


Index([u'sujet', u'niveau', u'm1_niveau', u'm11_score', u'm12_score', u'm2_niveau', u'm21_score', u'm22_score', u'm3_niveau', u'm31_score', u'm32_score', u'but_1', u'tirs_1', u'tirscadre_1', u'possession_1', u'tacles_1', u'fautes_1', u'cartonJaune_1', u'cartonRouge_1', u'blessure_1', u'horsjeu_1', u'corners_1', u'precisiontir_1', u'precisionpasses_1', u'but_2', u'tirs_2', u'tirscadre_2', u'possession_2', u'tacles_2', u'fautes_2', u'cartonJaune_2', u'cartonRouge_2', u'blessure_2', u'horsjeu_2', u'corners_2', u'precisiontir_2', u'precisionpasses_2', u'but_3', u'tirs_3', u'tirscadre_3', u'possession_3', u'tacles_3', u'fautes_3', u'cartonJaune_3', u'cartonRouge_3', u'blessure_3', u'horsjeu_3', u'corners_3', u'precisiontir_3', u'precisionpasses_3'], dtype='object')

In [19]:
mstadf.head()

,sujet,niveau,m1_niveau,m11_score,m12_score,m2_niveau,m21_score,m22_score,m3_niveau,m31_score,...,possession_3,tacles_3,fautes_3,cartonJaune_3,cartonRouge_3,blessure_3,horsjeu_3,corners_3,precisiontir_3,precisionpasses_3
0,1,1,2,0/1,0/1,1,0/0,0/1,2,0/0,...,48/52,2/4,0/0,0/0,0/0,0/0,0/0,0/0,100/0,78/74
1,2,4,3,0/0,0/0,5,0/0,0/1,2,1/0,...,48/52,1/1,0/0,0/0,0/0,0/0,2/0,0/0,100/0,90/85
2,3,2,4,0/3,0/4,2,0/0,0/0,1,0/0,...,49/51,3/1,0/0,0/0,0/0,0/0,0/0,0/0,0/50,82/70
3,4,3,4,0/0,0/0,2,2/0,4/0,3,0/1,...,48/52,0/2,0/0,0/0,0/0,0/0,0/1,1/0,100/66,83/83
4,5,5,5,0/0,0/0,4,0/0,1/0,6,0/0,...,56/44,4/3,1/1,0/0,0/0,0/0,0/0,2/0,33/50,89/91


In [20]:
def stacmp(score):
    if type(score) is str:
        our=int(score.split('/')[0])
        adver=int(score.split('/')[1])
        return (our-adver)/(our+adver+1.0)
    else:
        return np.nan

In [21]:
Matchinfo=mstadf[['sujet','niveau','m1_niveau','m2_niveau','m3_niveau']]
MatchStainfo=mstadf.drop(['sujet','niveau','m1_niveau','m2_niveau','m3_niveau'], axis=1)
MatchStainfo=MatchStainfo.applymap(stacmp)
MatchFeature = pd.concat([Matchinfo,MatchStainfo],axis=1)
MatchFeature.head()

,sujet,niveau,m1_niveau,m2_niveau,m3_niveau,m11_score,m12_score,m21_score,m22_score,m31_score,...,possession_3,tacles_3,fautes_3,cartonJaune_3,cartonRouge_3,blessure_3,horsjeu_3,corners_3,precisiontir_3,precisionpasses_3
0,1,1,2,1,2,-0.50,-0.5,0.000000,-0.5,0.0,...,-0.039604,-0.285714,0,0,0,0,0.000000,0.000000,0.990099,0.026144
1,2,4,3,5,2,0.00,0.0,0.000000,-0.5,0.5,...,-0.039604,0.000000,0,0,0,0,0.666667,0.000000,0.990099,0.028409
2,3,2,4,2,1,-0.75,-0.8,0.000000,0.0,0.0,...,-0.019802,0.400000,0,0,0,0,0.000000,0.000000,-0.980392,0.078431
3,4,3,4,2,3,0.00,0.0,0.666667,0.8,-0.5,...,-0.039604,-0.666667,0,0,0,0,-0.500000,0.500000,0.203593,0.000000
4,5,5,5,4,6,0.00,0.0,0.000000,0.5,0.0,...,0.118812,0.125000,0,0,0,0,0.000000,0.666667,-0.202381,-0.011050


#### Separate Match

In [22]:
Match1=MatchFeature[['sujet','niveau','m1_niveau','m11_score','m12_score','but_1', 'tirs_1', 'tirscadre_1', 'possession_1', 'tacles_1', 'fautes_1', 'cartonJaune_1', 'cartonRouge_1', 'blessure_1', 'horsjeu_1', 'corners_1', 'precisiontir_1', 'precisionpasses_1']]
Match1['match']=1
Match1.columns=['sujet','niveausub','niveaumat','butmi1','butmi2','but', 'tirs', 'tirscadre', 'possession', 'tacles', 'fautes', 'cartonJaune', 'cartonRouge', 'blessure', 'horsjeu', 'corners', 'precisiontir', 'precisionpasses','match']

Match2=MatchFeature[['sujet','niveau','m2_niveau','m21_score','m22_score','but_2', 'tirs_2', 'tirscadre_2', 'possession_2', 'tacles_2', 'fautes_2', 'cartonJaune_2', 'cartonRouge_2', 'blessure_2', 'horsjeu_2', 'corners_2', 'precisiontir_2', 'precisionpasses_2']]
Match2['match']=2
Match2.columns=['sujet','niveausub','niveaumat','butmi1','butmi2','but', 'tirs', 'tirscadre', 'possession', 'tacles', 'fautes', 'cartonJaune', 'cartonRouge', 'blessure', 'horsjeu', 'corners', 'precisiontir', 'precisionpasses','match']

Match3=MatchFeature[['sujet','niveau','m3_niveau','m31_score','m32_score','but_3', 'tirs_3', 'tirscadre_3', 'possession_3', 'tacles_3', 'fautes_3', 'cartonJaune_3', 'cartonRouge_3', 'blessure_3', 'horsjeu_3', 'corners_3', 'precisiontir_3', 'precisionpasses_3']]
Match3['match']=3
Match3.columns=['sujet','niveausub','niveaumat','butmi1','butmi2','but', 'tirs', 'tirscadre', 'possession', 'tacles', 'fautes', 'cartonJaune', 'cartonRouge', 'blessure', 'horsjeu', 'corners', 'precisiontir', 'precisionpasses','match']

MatchFeatureS=pd.concat([Match1,Match2,Match3],axis=0)
MatchFeatureS=MatchFeatureS.sort(['sujet','match']).reset_index(drop=True)
MatchFeatureS=MatchFeatureS.set_index(['sujet', 'match'])
MatchFeatureS.head()

niveausub  niveaumat  butmi1  butmi2  but  tirs  tirscadre  \
sujet match                                                               
1     1              1          2    -0.5    -0.5 -0.5  0.00   0.000000   
      2              1          1     0.0    -0.5  0.0 -0.40   0.500000   
      3              1          2     0.0     0.5  0.0 -0.40   0.500000   
2     1              4          3     0.0     0.0  0.0  0.00   0.500000   
      2              4          5     0.0    -0.5  0.0 -0.75  -0.666667   

             possession    tacles    fautes  cartonJaune  cartonRouge  \
sujet match                                                             
1     1       -0.178218  0.000000 -0.500000          0.0            0   
      2       -0.178218 -0.800000  0.000000          0.0            0   
      3       -0.039604 -0.285714  0.000000          0.0            0   
2     1       -0.178218  0.000000  0.000000          0.0            0   
      2       -0.178218  0.250000  0.666667          0.5            0   

             blessure   horsjeu   corners  precisiontir  precisionpasses  
sujet match                                                               
1     1             0  0.000000  0.000000      0.000000        -0.229167  
      2             0  0.000000  0.000000      0.990099         0.000000  
      3             0  0.000000  0.000000      0.990099         0.026144  
2     1             0  0.666667  0.000000      0.990099        -0.029070  
      2             0  0.000000 -0.666667     -0.985075        -0.124224

In [23]:
MatchFeatureS.to_csv('./../MatchFeature.csv',encoding='utf-16')

# Combine features with evaluation

### Matchfeature

In [24]:
MatchFeatureS=MatchFeatureS.convert_objects(convert_numeric=True)
MatchFeatureS.head()

niveausub  niveaumat  butmi1  butmi2  but  tirs  tirscadre  \
sujet match                                                               
1     1              1          2    -0.5    -0.5 -0.5  0.00   0.000000   
      2              1          1     0.0    -0.5  0.0 -0.40   0.500000   
      3              1          2     0.0     0.5  0.0 -0.40   0.500000   
2     1              4          3     0.0     0.0  0.0  0.00   0.500000   
      2              4          5     0.0    -0.5  0.0 -0.75  -0.666667   

             possession    tacles    fautes  cartonJaune  cartonRouge  \
sujet match                                                             
1     1       -0.178218  0.000000 -0.500000          0.0            0   
      2       -0.178218 -0.800000  0.000000          0.0            0   
      3       -0.039604 -0.285714  0.000000          0.0            0   
2     1       -0.178218  0.000000  0.000000          0.0            0   
      2       -0.178218  0.250000  0.666667          0.5            0   

             blessure   horsjeu   corners  precisiontir  precisionpasses  
sujet match                                                               
1     1             0  0.000000  0.000000      0.000000        -0.229167  
      2             0  0.000000  0.000000      0.990099         0.000000  
      3             0  0.000000  0.000000      0.990099         0.026144  
2     1             0  0.666667  0.000000      0.990099        -0.029070  
      2             0  0.000000 -0.666667     -0.985075        -0.124224

### Event feature

In [25]:
EventFeature=pd.read_csv('/home/wenlu/Documents/XPFIFA/EventFeature.csv',encoding='utf-16',index_col=[0,1])
EventFeature=EventFeature.fillna(0)
EventFeature=EventFeature.convert_objects(convert_numeric=True)
EventFeature.head()

all  vapos  vaneg  fortva  arpos  arneg  forar  frus  colere  \
sujet match                                                                 
1     1        4      0      4       1      4      0      0     1       2   
      2        6      0      6       0      5      1      0     3       1   
      3        8      1      7       0      8      0      0     2       1   
2     1        9      2      7       3      8      1      2     5       2   
      2       17      2     15       1     14      3      2     7       5   

             joie  
sujet match        
1     1         0  
      2         0  
      3         0  
2     1         0  
      2         1

### Physio feature

In [26]:
PhysioMatFeatureNorm=pd.read_csv('/home/wenlu/Documents/XPFIFA/PhysioMatFeatureNorm.csv',encoding='utf-16',index_col=0)
PhysioWhole=PhysioMatFeatureNorm.ix[(PhysioMatFeatureNorm['Nummitemp']==2)&(PhysioMatFeatureNorm['NumMatch'].isin([1,2,3]))]
PhysioWhole=PhysioWhole.rename(index=str, columns={'NumMatch':'match',})
PhysioWhole=PhysioWhole.drop(['sequence','Nummitemp'],axis=1).set_index(['sujet','match'])
PhysioWhole=PhysioWhole.convert_objects(convert_numeric=True)
PhysioWhole.head()

hbd_mean  hbd_median   hbd_max   hbd_min  hbd_vrange   hbd_var  \
sujet match                                                                   
2     1     -0.202853   -0.218763 -0.287746 -0.107739   -0.315017 -0.848462   
      2      0.186026    0.168116  0.347295  0.267647    0.292991 -0.043301   
      3      1.417388    1.439289  1.736448  1.060128    1.641183  0.901195   
3     1     -0.589604   -0.718126  0.607157 -0.811482    0.731647  0.248784   
      2     -0.274262   -0.231045 -1.075315  0.769937   -1.064764 -0.741153   

             hbd_std_dev  hbd_avg_der  hbd_max_gra  hbd_abs_dev     ...       \
sujet match                                                         ...        
2     1        -0.840774     0.705649     0.609294    -0.523872     ...        
      2         0.067413     0.442530    -0.241782     0.447733     ...        
      3         0.927256    -0.460211    -0.125726     1.001798     ...        
3     1         0.388393     0.365048     0.651403     1.239751     ...        
      2        -0.845734     0.057843    -1.092748    -0.127658     ...        

               rsp_b0    rsp_b1    rsp_b2    rsp_b3    rsp_b4  rsprt_b0  \
sujet match                                                               
2     1     -0.765506 -0.735363 -0.670909 -0.901918 -0.986010  1.078812   
      2     -0.533039 -0.462436 -0.505334 -0.844468 -0.483609 -1.372149   
      3      1.430893  1.434551  1.215246  1.064914  1.023238 -1.318026   
3     1     -0.465081 -0.495199 -0.340517 -0.207278 -0.435492  0.450745   
      2     -0.168673 -0.071239 -0.184005  0.616904 -0.276543  0.464168   

             rsprt_b1  rsprt_b2  rsprt_b3  rsprt_b4  
sujet match                                          
2     1      1.078812       NaN       NaN       NaN  
      2     -1.372149       NaN       NaN       NaN  
      3     -1.318026       NaN       NaN       NaN  
3     1      0.450745       NaN       NaN       NaN  
      2      0.464168       NaN       NaN       NaN  

[5 rows x 100 columns]

### Evaluation Label

In [27]:
rankevadf.head()

,sujet,seqlabel,NoMatch,Nomitemps,noteg,change,eventchange,noteI,noteD,noteA,...,rankgI,rankgD,rankgA,rdiffqI,rdiffqD,rdiffqA,rdiffnI,rdiffnD,rdiffnA,sujetNum
0,S3,02,0,2,4,1,0,5,5,2,...,2,2,2,0.5,1,2,0.5,1.0,1.5,3
1,S3,12,1,2,6,1,0,5,4,4,...,1,1,1,0.5,1,1,0.5,1.5,0.0,3
2,S3,22,2,2,4,1,1,4,4,2,...,4,4,4,0.0,1,1,0.5,1.5,0.5,3
3,S3,32,3,2,7,1,1,4,3,3,...,3,3,3,0.0,1,2,0.5,1.0,1.0,3
4,S4,02,0,2,7,-1,1,3,3,4,...,4,4,4,0.5,1,0,0.0,1.0,0.5,4


In [28]:
rankWhole=rankevadf.drop(['sujet','seqlabel','Nomitemps'],axis=1)
rankWhole=rankWhole.ix[rankWhole['NoMatch']!='0',:]
rankWhole['NoMatch']=rankWhole['NoMatch'].astype(int)
rankWhole=rankWhole.rename(index=str,columns={'sujetNum':'sujet','NoMatch':'match'}).set_index(['sujet','match'])
rankWhole=rankWhole.convert_objects(convert_numeric=True)

In [29]:
rankWhole.head()

noteg  change  eventchange  noteI  noteD  noteA   qI   qD   qA  \
sujet match                                                                   
3     1          6       1            0      5      4      4  5.0  3.5  3.5   
      2          4       1            1      4      4      2  4.0  3.0  2.5   
      3          7       1            1      4      3      3  4.5  2.5  4.0   
4     1          8       1            1      4      5      5  4.0  4.5  4.0   
      2          8      -1            1      5      2      4  4.5  1.0  4.0   

             rnI ...   rqA  rankgI  rankgD  rankgA  rdiffqI  rdiffqD  rdiffqA  \
sujet match      ...                                                            
3     1      1.5 ...     2       1       1       1      0.5        1        1   
      2      3.5 ...     3       4       4       4      0.0        1        1   
      3      3.5 ...     1       3       3       3      0.0        1        2   
4     1      3.0 ...     2       3       3       3      0.5        2        1   
      2      1.5 ...     2       2       2       2      0.5        2        0   

             rdiffnI  rdiffnD  rdiffnA  
sujet match                             
3     1          0.5      1.5      0.0  
      2          0.5      1.5      0.5  
      3          0.5      1.0      1.0  
4     1          0.0      2.0      1.5  
      2          0.5      2.0      1.5  

[5 rows x 24 columns]

In [30]:
allmerge= rankWhole.join([PhysioWhole,EventFeature,MatchFeatureS])

In [31]:
dataset=allmerge[6::]
list(dataset.columns.values)

['noteg',
 'change',
 'eventchange',
 'noteI',
 'noteD',
 'noteA',
 'qI',
 'qD',
 'qA',
 'rnI',
 'rnD',
 'rnA',
 'rqI',
 'rqD',
 'rqA',
 'rankgI',
 'rankgD',
 'rankgA',
 'rdiffqI',
 'rdiffqD',
 'rdiffqA',
 'rdiffnI',
 'rdiffnD',
 'rdiffnA',
 u'hbd_mean',
 u'hbd_median',
 u'hbd_max',
 u'hbd_min',
 u'hbd_vrange',
 u'hbd_var',
 u'hbd_std_dev',
 u'hbd_avg_der',
 u'hbd_max_gra',
 u'hbd_abs_dev',
 u'hbd_kurtosis',
 u'hbd_skewness',
 u'hr_mean',
 u'hr_median',
 u'hr_max',
 u'hr_min',
 u'hr_vrange',
 u'hr_var',
 u'hr_std_dev',
 u'hr_avg_der',
 u'hr_max_gra',
 u'hr_abs_dev',
 u'hr_kurtosis',
 u'hr_skewness',
 u'hr_b0',
 u'hr_b1',
 u'hr_b2',
 u'hr_b3',
 u'hr_b4',
 u'ecg_b0',
 u'ecg_b1',
 u'ecg_b2',
 u'ecg_b3',
 u'ecg_b4',
 u'emgct_f',
 u'emgct_f_s',
 u'scramps_mean',
 u'scramps_median',
 u'scramps_max',
 u'scramps_min',
 u'scramps_vrange',
 u'scramps_var',
 u'scramps_std_dev',
 u'scramps_avg_der',
 u'scramps_max_gra',
 u'scramps_abs_dev',
 u'scramps_kurtosis',
 u'scramps_skewness',
 u'eda_ct',
 

In [55]:
from sklearn.ensemble import RandomForestClassifier
X=np.array(datasetv.iloc[:,24::])
Y=np.array(datasetv['change'])
clf = RandomForestClassifier(n_estimators=10)
clf = clf.fit(X, Y)


In [51]:
datasetv=dataset.drop([ 'scramps_mean',   'scramps_median',   'scramps_max',   'scramps_min',   'scramps_vrange',   'scramps_var',   'scramps_std_dev',   'scramps_avg_der',   'scramps_max_gra',   'scramps_abs_dev',   'scramps_kurtosis',   'scramps_skewness','eda_ct','rsprt_b2','rsprt_b3','rsprt_b4'], axis=1)

In [54]:
datasetv=datasetv.dropna()